# Training Model

In [ ]:
from core.networks import AE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

import copy
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
preprocess = {
    "train": transforms.Compose([
        transforms.Resize(64),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
    ]),
    "val": transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
    ])
}

In [ ]:
datasets = {
    x: CIFAR10(
        root="./data",
        train = True if x == "train" else False,
        transform = preprocess[x],
    )
    for x in ["train", "val"]
}

In [ ]:
datasets["train"]

In [ ]:
datasets["val"]

In [ ]:
dataloaders = {
    x: DataLoader(
        datasets[x],
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    for x in ["train", "val"]
}

dataset_sizes = {
    x: len(datasets[x])
    for x in ["train", "val"]
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def imshow(inp, title=None):
    """ Imshow for Tensor """
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp, cmap=plt.cm.gray)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
inputs, _ = next(iter(dataloaders["train"]))
out = torchvision.utils.make_grid(inputs[:16])
imshow(out)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    loss_tracker = {
        "train": [],
        "val": [],
    }

    for epoch in range(num_epochs):
        start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, _ in dataloaders[phase]:

                inputs = inputs.to(device)
                # targets = targets.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, inputs)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / dataset_sizes[phase]

            loss_tracker[phase].append(epoch_loss)
            acc_tracker[phase].append(epoch_acc)

            print('{} Loss: {:.4f} - Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        epoch_run_time = time.time() - start
        print(f'epoch running time: {epoch_run_time//60:.03}m {epoch_run_time%60:.03}s')
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_tracker

In [ ]:
model = AE().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model, losses = train_model(
    model,
    criterion,
    optimizer,
    num_epochs=2,
) 